In [1]:
import tarfile
from itertools import groupby
from zlib import decompress
import functools

import numpy as np
import zstandard

In [2]:
import os
from pathlib import Path

In [3]:
from datetime import datetime
import polars as pl
import glob
from tqdm import tqdm

In [4]:
metadata = pl.read_csv('raw/metadata.csv', try_parse_dates=True)
metadata

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""00000c5a448d9faa097b761cc98036…",2022-05-30 01:00:00 UTC,2022-06-05 00:00:00 UTC,6.0,0.016427,144,144,0,0.0,2022-05-31,null,"""2.0TD""","""""",2.2,2.2,0.0,0.0,0.0,0.0,"""Gipuzkoa""",null,null,9329
"""0001b3b2f18c01c62ed9b2a87de7b4…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43863,81,0.184326,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Bizkaia""",null,null,9820
"""0003de2700e20a1681d69fe287441d…",2017-05-31 01:00:00 UTC,2019-11-14 00:00:00 UTC,897.0,2.455852,21528,21476,52,0.241546,2019-09-02,2019-11-13,"""2.0A""","""""",4.6,null,null,null,null,null,"""Gipuzkoa""","""Donostia/San Sebastian""",20013,9820
"""0004150214d14a2b2e6f7075531e66…",2018-07-12 01:00:00 UTC,2022-06-05 00:00:00 UTC,1424.0,3.8987,34176,34169,7,0.020482,2021-06-01,null,"""2.0TD""","""""",9.2,9.2,null,null,null,null,"""Gipuzkoa""","""Irun""",20304,4759
"""000721f0fc6ccf02ae24b673939795…",2017-05-30 01:00:00 UTC,2022-06-04 00:00:00 UTC,1831.0,5.013005,43944,43815,129,0.293555,2021-06-01,null,"""2.0TD""","""""",4.4,4.4,null,null,null,null,"""Bizkaia""",null,null,9820
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ffeddf4919e748beb50af7c6b73def…",2018-10-02 01:00:00 UTC,2021-11-03 00:00:00 UTC,1128.0,3.088296,27072,27069,3,0.011082,2021-06-01,null,"""2.0TD""","""""",6.0,6.0,null,null,null,null,"""Soria""",null,null,9820
"""ffefb9e27715b1c3fa5007c457de55…",2021-07-06 01:00:00 UTC,2022-06-05 00:00:00 UTC,334.0,0.914442,8016,8014,2,0.02495,2021-07-07,null,"""2.0TD""","""""",3.45,3.45,0.0,0.0,0.0,0.0,"""Bizkaia""","""Bilbao""",48012,9820
"""fff0adcd6cdb7434496a5222f03337…",2017-08-17 01:00:00 UTC,2022-06-05 00:00:00 UTC,1753.0,4.799452,42072,41102,970,2.305571,2021-06-01,null,"""2.0TD""","""""",4.6,4.6,null,null,null,null,"""Bizkaia""",null,null,9820


In [5]:
metadata_pst_house_2 = metadata.filter(
    pl.col("end_date").is_between(pl.datetime(2022, 5, 30, time_zone="UTC"), pl.datetime(3000, 1, 1, time_zone="UTC")),
    pl.col("length_years") >= 1,
    # (pl.col('cnae') > 9699),
    # (pl.col('cnae') < 9849)
)
metadata_pst_house_2.head()

user,start_date,end_date,length_days,length_years,potential_samples,actual_samples,missing_samples_abs,missing_samples_pct,contract_start_date,contract_end_date,contracted_tariff,self_consumption_type,p1,p2,p3,p4,p5,p6,province,municipality,zip_code,cnae
str,"datetime[μs, UTC]","datetime[μs, UTC]",f64,f64,i64,i64,i64,f64,date,date,str,str,f64,f64,f64,f64,f64,f64,str,str,i64,i64
"""0001b3b2f18c01c62ed9b2a87de7b4…",2017-05-31 01:00:00 UTC,2022-06-05 00:00:00 UTC,1831.0,5.013005,43944,43863,81,0.184326,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Bizkaia""",null,null,9820
"""0004150214d14a2b2e6f7075531e66…",2018-07-12 01:00:00 UTC,2022-06-05 00:00:00 UTC,1424.0,3.8987,34176,34169,7,0.020482,2021-06-01,null,"""2.0TD""","""""",9.2,9.2,null,null,null,null,"""Gipuzkoa""","""Irun""",20304,4759
"""000721f0fc6ccf02ae24b673939795…",2017-05-30 01:00:00 UTC,2022-06-04 00:00:00 UTC,1831.0,5.013005,43944,43815,129,0.293555,2021-06-01,null,"""2.0TD""","""""",4.4,4.4,null,null,null,null,"""Bizkaia""",null,null,9820
"""0009b156f2a1213a137c150f478715…",2017-12-11 01:00:00 UTC,2022-06-04 00:00:00 UTC,1636.0,4.479124,39264,39255,9,0.022922,2021-06-01,null,"""2.0TD""","""""",3.45,3.45,null,null,null,null,"""Gipuzkoa""",null,null,9820
"""000bf84faacf921b55bd4ec4aecda5…",2019-03-07 01:00:00 UTC,2022-06-05 00:00:00 UTC,1186.0,3.247091,28464,28457,7,0.024592,2021-06-01,null,"""2.0TD""","""""",9.2,9.2,null,null,null,null,"""Gipuzkoa""","""Donostia/San Sebastian""",20003,9820


In [6]:
metadata_pst_house_2_users = metadata_pst_house_2['user'].unique().to_list()

In [7]:
print(f"Post-COVID household users={len(metadata_pst_house_2_users)}")

Post-COVID household users=17655


# Combine CSV

In [32]:
source_folder = 'dataset/goi4_pst/imp_csv'
csv_files = glob.glob(f"{source_folder}/*.csv")

In [44]:
def combine(mask: list[str], csv_files: list[str], filename:str='dataset/goi4_pst/household_activities_all.csv') -> pl.DataFrame:
    """
    Combine multiple CSV files into a single DataFrame, filtering by household profiles.
    """
    dataframes = pl.DataFrame()
    for csv_file in tqdm(csv_files):
        profile_id = csv_file.split('/')[-1].split('.')[0]
        if profile_id not in mask:
            continue

        df = pl.read_csv(csv_file, try_parse_dates=True, schema_overrides={'kWh':pl.Float64})
        df = df.with_columns(pl.lit(profile_id).alias('profile_id'))
        if not dataframes.columns:
            dataframes = df.clone()
            continue
        dataframes.extend(df)
    if filename:
        dataframes.write_csv(filename)

    return dataframes

In [45]:
combine(metadata_pst_house_2_users, csv_files)

100%|██████████| 17519/17519 [00:33<00:00, 515.64it/s]


timestamp,kWh,imputed,profile_id
datetime[μs],f64,i64,str
2021-05-31 00:00:00,0.133,0,"""e09da31b7a07f3f97c51c78dea39f9…"
2021-05-31 01:00:00,0.125,0,"""e09da31b7a07f3f97c51c78dea39f9…"
2021-05-31 02:00:00,0.184,0,"""e09da31b7a07f3f97c51c78dea39f9…"
2021-05-31 03:00:00,0.137,0,"""e09da31b7a07f3f97c51c78dea39f9…"
2021-05-31 04:00:00,0.111,0,"""e09da31b7a07f3f97c51c78dea39f9…"
…,…,…,…
2022-06-04 20:00:00,0.057,0,"""5793ddb25829ba3db566b882d7cf11…"
2022-06-04 21:00:00,0.046,0,"""5793ddb25829ba3db566b882d7cf11…"
2022-06-04 22:00:00,0.049,0,"""5793ddb25829ba3db566b882d7cf11…"


In [46]:
df_activities = pl.read_csv("dataset/goi4_pst/household_activities_all.csv")
df_activities.head()

timestamp,kWh,imputed,profile_id
str,f64,i64,str
"""2021-05-31T00:00:00.000000""",0.133,0,"""e09da31b7a07f3f97c51c78dea39f9…"
"""2021-05-31T01:00:00.000000""",0.125,0,"""e09da31b7a07f3f97c51c78dea39f9…"
"""2021-05-31T02:00:00.000000""",0.184,0,"""e09da31b7a07f3f97c51c78dea39f9…"
"""2021-05-31T03:00:00.000000""",0.137,0,"""e09da31b7a07f3f97c51c78dea39f9…"
"""2021-05-31T04:00:00.000000""",0.111,0,"""e09da31b7a07f3f97c51c78dea39f9…"


In [1]:
df_activities['profile_id'].unique()

NameError: name 'df_activities' is not defined